In [14]:
import os
import shutil
import matplotlib.pyplot as plt
import glob
from create_dataset_utils import *
import numpy as np


In [12]:
def read_yolo_boxes(label_file, img_width, img_height):
    boxes = []
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue  # skip lable không đủ value [id, x_center, y_center, width, height]
            # convert về [x1, y1, x2, y2]
            _, x_center_norm, y_center_norm, width_norm, height_norm = parts
            x_center = float(x_center_norm) * img_width
            y_center = float(y_center_norm) * img_height
            box_width = float(width_norm) * img_width
            box_height = float(height_norm) * img_height
            x1 = x_center - box_width / 2
            y1 = y_center - box_height / 2
            x2 = x_center + box_width / 2
            y2 = y_center + box_height / 2
            boxes.append([x1, y1, x2, y2])
    return boxes

**Tạo train dataset(odvg format)**

In [ ]:
# Define directory paths
input_folder = r'C:\Users\phann\Documents\Pig_farming\dataset\pigs_detection\train'
output_dir = r'C:\Users\phann\Documents\Pig_farming\dataset\output_dataset_no_scale\train' 

# Create output directories
images_output_dir = os.path.join(output_dir, "images")
os.makedirs(images_output_dir, exist_ok=True)

# Cập nhật frame counter từ file
existing_images = glob.glob(os.path.join(images_output_dir, "*.jpg"))
existing_numbers = [
    int(os.path.basename(f).split('_')[0])
    for f in existing_images
    if os.path.basename(f).split('_')[0].isdigit()
]
global_frame_counter = max(existing_numbers) + 1 if existing_numbers else 1

# Load annotations 
annotation_file = os.path.join(output_dir, "annotation.json")
if os.path.exists(annotation_file):
    with open(annotation_file, 'r') as f:
        annotations = json.load(f)
else:
    annotations = os.path.join(output_dir, "annotations.jsonl")

# Load categories
categories_file = os.path.join(output_dir, "categories.json")
if os.path.exists(categories_file):
    with open(categories_file, 'r') as f:
        categories = json.load(f)
else:
    categories = {
        "categories": [
            {"id": 0, "name": "pig"}
        ]
    }

print(f"Starting frame counter: {global_frame_counter}")

scales = [None] # list các scale tỉ lệ để resize ảnh

# Lấy file ảnh và label
image_files = glob.glob(os.path.join(input_folder, "**/*.jpg"), recursive=True)
label_files = glob.glob(os.path.join(input_folder, "**/*.txt"), recursive=True)

label_dict = {}
for lf in label_files:
    base_name = os.path.splitext(os.path.basename(lf))[0]
    label_dict[base_name] = lf

for image_path in image_files:
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Check matching label file
    if base_name not in label_dict:
        print(f"No label file found for {image_path}. Skipping.")
        continue

    current_label_file = label_dict[base_name]

    label_base = os.path.splitext(os.path.basename(current_label_file))[0]
    if base_name != label_base:
        print(f"Mismatch between image and label names: {image_path} and {current_label_file}. Skipping.")
        continue

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}. Skipping.")
        continue

    img_h, img_w = image.shape[:2]

    # Extract bounding boxes from the YOLO-format label file
    boxes = read_yolo_boxes(current_label_file, img_w, img_h)
    if not boxes:
        print(f"No boxes found in {current_label_file}. Skipping.")
        continue
    
    # Create dataset
    global_frame_counter, annotations = create_dataset_GroundDino_odvg(
        image, boxes, categories, output_dir, annotations, global_frame_counter, scales=scales
    )
    print(f"Processed dataset for image: {image_path}")

print("Dataset creation complete!")


Starting frame counter: 1
No label file found for C:\Users\phann\Documents\project\dataset\pigs_detection\train\20190624141650_original.jpg. Skipping.
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1001_jpg.rf.2ff560c48e22d3b870ef5600ba5cd2a7.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1002_jpg.rf.99aa967d02ce80f7a1a76e19805c466b.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1003_jpg.rf.c72227cd2fcc9f6f4427d5b733df6c4b.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1004_jpg.rf.a83ee8c490167f3c31adc4f1ebbe8ab9.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1005_jpg.rf.1c93bb601859699fa2eac9899a2f2e45.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1006_jpg.rf.8ee9

**summary dataset key**

In [ ]:

json_file_path = r"C:\Users\phann\Documents\Pig_farming\dataset\coco_test_exemplars.json"

# Load the JSON data from the file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Xem datast format
for top_key, value in data.items():
    if isinstance(value, list) and value and isinstance(value[0], dict):
        print(f"\nSummary for key: '{top_key}'")
        for idx, entry in enumerate(value, start=1):
            print(f"Entry {entry}:")
            break


Summary for key: 'images'
Entry {'height': 384, 'width': 633, 'id': 1, 'file_name': '2.jpg'}:

Summary for key: 'annotations'
Entry {'iscrowd': 0, 'image_id': 1, 'bbox': [48.2078375, 242.82812371134017, 2, 2], 'category_id': 21, 'id': 1, 'area': 4}:

Summary for key: 'categories'
Entry {'id': 0, 'name': 'apple'}:


**Tạo val dataset( COCO format)**

In [27]:

# Define directory paths
input_folder = r'C:\Users\phann\Documents\project\dataset\pigs_detection\valid'
output_dir = r'C:\Users\phann\Documents\project\dataset\output_dataset_no_scale\valid'  # Where processed outputs will be saved

# Create output directories 
images_output_dir = os.path.join(output_dir, "images")
os.makedirs(images_output_dir, exist_ok=True)

# Load existing images and annotations 
existing_images = glob.glob(os.path.join(images_output_dir, "*.jpg"))
existing_numbers = [
    int(os.path.basename(f).split('_')[0])
    for f in existing_images
    if os.path.basename(f).split('_')[0].isdigit()
]
global_frame_counter = max(existing_numbers) + 1 if existing_numbers else 1

# Load annotations
annotation_file = os.path.join(output_dir, "annotation.json")
if os.path.exists(annotation_file):
    with open(annotation_file, 'r') as f:
        coco_dataset = json.load(f)
else:
    # COCO dataset structure main key
    coco_dataset = {"images": [], "annotations": [], "categories": []}

categories = [
        {"id": 0, "name": "pig"},
    ]


# Lấy file ảnh và label
image_files = glob.glob(os.path.join(input_folder, "**/*.jpg"), recursive=True)
label_files = glob.glob(os.path.join(input_folder, "**/*.txt"), recursive=True)

# Tạo dictionary để map image và label
label_dict = {}
for lf in label_files:
    base_name = os.path.splitext(os.path.basename(lf))[0]
    label_dict[base_name] = lf

processed_image_paths = []
processed_boxes = []

for image_path in image_files:
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    # Check matching label file
    if base_name not in label_dict:
        print(f"No label file found for {image_path}. Skipping.")
        continue

    current_label_file = label_dict[base_name]

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}. Skipping.")
        continue

    img_h, img_w = image.shape[:2]

    # Extract bounding boxes 
    boxes = read_yolo_boxes(current_label_file, img_w, img_h)
    if not boxes:
        print(f"No boxes found in {current_label_file}. Skipping.")
        continue
    
    if len(boxes) < 3:
            print(f"Image {image_path} has less than 3 boxes. Skipping.")
            continue

    if base_name != os.path.splitext(os.path.basename(current_label_file))[0]:
        print(f"Mismatch between image and label names: {image_path} and {current_label_file}. Skipping.")
        continue

    processed_image_paths.append(image_path)
    processed_boxes.append(boxes)
    print(f"Image and label match confirmed: {base_name}")
    
# Create dataset
coco_dataset, new_frame_counter = create_dataset_GroundDino_coco(
    processed_image_paths, processed_boxes, output_dir, categories, global_frame_counter, scales=[None]
)

# Save updated annotations (COCO dataset) to file
with open(annotation_file, 'w') as f:
    json.dump(coco_dataset, f, indent=4)

print("Dataset creation complete!")


Image and label match confirmed: 1010s1120s2001-2s5300-1-1126_jpg.rf.5438a4594cbd4d31ecbc973b81c6212d
Image and label match confirmed: 1010s1120s2001-2s5300-1-451_jpg.rf.999dd4899cd03c61218b66fb0dfae86f
Image and label match confirmed: 1010s1120s2001-2s5300-1-526_jpg.rf.7fed3d5b233ef4016ad8a40557d0f233
Image and label match confirmed: 1010s1120s2001-2s5300-1-556_jpg.rf.4f82fd6fb48861f28f66b487c40fed80
Image and label match confirmed: 1010s1120s2001-2s5300-1-631_jpg.rf.507be1b21e153ff617c6e8d5dcef4835
Image and label match confirmed: 1010s1120s2001-2s5300-1-676_jpg.rf.074926106e5d1f7d9e99c79e1f04e666
Image and label match confirmed: 1010s1120s2001-2s5300-1-886_jpg.rf.0e1c01ae629d6402ecaf54f676166fbb
Image and label match confirmed: 1010s1121s2001-2s5101-1-136_jpg.rf.1207b4095ecca7dbac948afcf9814ebf
Image and label match confirmed: 1010s1121s2001-2s5101-1-211_jpg.rf.c23c480cf8ad5706dde13cbebae229e1
Image and label match confirmed: 1010s1121s2001-2s5101-1-271_jpg.rf.11003295413988ccd4adaf

In [ ]:
# Define directory paths
input_folder = r'C:\Users\phann\Documents\project\dataset\pigs_detection\train'
output_dir = r'C:\Users\phann\Documents\project\dataset\output_dataset_test\train' 

# Create output directories
images_output_dir = os.path.join(output_dir, "images")
os.makedirs(images_output_dir, exist_ok=True)

# Cập nhật frame counter từ file
existing_images = glob.glob(os.path.join(images_output_dir, "*.jpg"))
existing_numbers = [
    int(os.path.basename(f).split('_')[0])
    for f in existing_images
    if os.path.basename(f).split('_')[0].isdigit()
]
global_frame_counter = max(existing_numbers) + 1 if existing_numbers else 1

# Load annotations 
annotation_file = os.path.join(output_dir, "annotation.json")
if os.path.exists(annotation_file):
    with open(annotation_file, 'r') as f:
        annotations = json.load(f)
else:
    annotations = os.path.join(output_dir, "annotations.jsonl")

# Load categories
categories_file = os.path.join(output_dir, "categories.json")
if os.path.exists(categories_file):
    with open(categories_file, 'r') as f:
        categories = json.load(f)
else:
    categories = {
        "categories": [
            {"id": 0, "name": "pig"}
        ]
    }

print(f"Starting frame counter: {global_frame_counter}")

scales = [None] # list các scale tỉ lệ để resize ảnh

# Lấy file ảnh và label
image_files = glob.glob(os.path.join(input_folder, "**/*.jpg"), recursive=True)
label_files = glob.glob(os.path.join(input_folder, "**/*.txt"), recursive=True)

label_dict = {}
for lf in label_files:
    base_name = os.path.splitext(os.path.basename(lf))[0]
    label_dict[base_name] = lf

for image_path in image_files:
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Check matching label file
    if base_name not in label_dict:
        print(f"No label file found for {image_path}. Skipping.")
        continue

    current_label_file = label_dict[base_name]

    label_base = os.path.splitext(os.path.basename(current_label_file))[0]
    if base_name != label_base:
        print(f"Mismatch between image and label names: {image_path} and {current_label_file}. Skipping.")
        continue

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}. Skipping.")
        continue

    img_h, img_w = image.shape[:2]

    # Extract bounding boxes from the YOLO-format label file
    boxes = read_yolo_boxes(current_label_file, img_w, img_h)
    if not boxes:
        print(f"No boxes found in {current_label_file}. Skipping.")
        continue
    
    # Create dataset
    global_frame_counter, annotations = create_dataset_GroundDino_odvg(
        image, boxes, categories, output_dir, annotations, global_frame_counter, scales=scales
    )
    print(f"Processed dataset for image: {image_path}")
    if global_frame_counter==100:
        break
print("Dataset creation complete!")


Starting frame counter: 1
No label file found for C:\Users\phann\Documents\project\dataset\pigs_detection\train\20190624141650_original.jpg. Skipping.
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1001_jpg.rf.2ff560c48e22d3b870ef5600ba5cd2a7.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1002_jpg.rf.99aa967d02ce80f7a1a76e19805c466b.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1003_jpg.rf.c72227cd2fcc9f6f4427d5b733df6c4b.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1004_jpg.rf.a83ee8c490167f3c31adc4f1ebbe8ab9.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1005_jpg.rf.1c93bb601859699fa2eac9899a2f2e45.jpg
Processed dataset for image: C:\Users\phann\Documents\project\dataset\pigs_detection\train\images\1006_jpg.rf.8ee9

In [25]:
import os
import glob
import json
import cv2

# Define directory paths
input_folder = r'C:\Users\phann\Documents\project\dataset\pigs_detection\valid'
output_dir = r'C:\Users\phann\Documents\project\dataset\output_dataset_test\valid'  # Where processed outputs will be saved

# Create output directories 
images_output_dir = os.path.join(output_dir, "images")
os.makedirs(images_output_dir, exist_ok=True)

# Load existing images and annotations 
existing_images = glob.glob(os.path.join(images_output_dir, "*.jpg"))
existing_numbers = [
    int(os.path.basename(f).split('_')[0])
    for f in existing_images
    if os.path.basename(f).split('_')[0].isdigit()
]
global_frame_counter = max(existing_numbers) + 1 if existing_numbers else 1

# Load annotations
annotation_file = os.path.join(output_dir, "annotation.json")
if os.path.exists(annotation_file):
    with open(annotation_file, 'r') as f:
        coco_dataset = json.load(f)
else:
    # COCO dataset structure main key
    coco_dataset = {"images": [], "annotations": [], "categories": []}

categories = [
    {"id": 0, "name": "pig"},
]

# Get image and label files
image_files = glob.glob(os.path.join(input_folder, "**/*.jpg"), recursive=True)
label_files = glob.glob(os.path.join(input_folder, "**/*.txt"), recursive=True)

# Create dictionary to map images to labels
label_dict = {}
for lf in label_files:
    base_name = os.path.splitext(os.path.basename(lf))[0]
    label_dict[base_name] = lf

processed_image_paths = []
processed_boxes = []

# Process only the first 50 images
for image_path in image_files[:50]:
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Check matching label file
    if base_name not in label_dict:
        print(f"No label file found for {image_path}. Skipping.")
        continue

    current_label_file = label_dict[base_name]

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}. Skipping.")
        continue

    img_h, img_w = image.shape[:2]

    # Extract bounding boxes 
    boxes = read_yolo_boxes(current_label_file, img_w, img_h)
    if not boxes:
        print(f"No boxes found in {current_label_file}. Skipping.")
        continue

    # Skip images with fewer than 3 bounding boxes
    if len(boxes) < 3:
        print(f"Image {image_path} has less than 3 boxes. Skipping.")
        continue

    if base_name != os.path.splitext(os.path.basename(current_label_file))[0]:
        print(f"Mismatch between image and label names: {image_path} and {current_label_file}. Skipping.")
        continue

    processed_image_paths.append(image_path)
    processed_boxes.append(boxes)
    print(f"Image and label match confirmed: {base_name}")
    
# Create dataset
coco_dataset, new_frame_counter = create_dataset_GroundDino_coco(
    processed_image_paths, processed_boxes, output_dir, categories, global_frame_counter, scales=[None]
)

# Save updated annotations (COCO dataset) to file
with open(annotation_file, 'w') as f:
    json.dump(coco_dataset, f, indent=4)

print("Dataset creation complete!")


Image and label match confirmed: 1010s1120s2001-2s5300-1-1126_jpg.rf.5438a4594cbd4d31ecbc973b81c6212d
Image and label match confirmed: 1010s1120s2001-2s5300-1-451_jpg.rf.999dd4899cd03c61218b66fb0dfae86f
Image and label match confirmed: 1010s1120s2001-2s5300-1-526_jpg.rf.7fed3d5b233ef4016ad8a40557d0f233
Image and label match confirmed: 1010s1120s2001-2s5300-1-556_jpg.rf.4f82fd6fb48861f28f66b487c40fed80
Image and label match confirmed: 1010s1120s2001-2s5300-1-631_jpg.rf.507be1b21e153ff617c6e8d5dcef4835
Image and label match confirmed: 1010s1120s2001-2s5300-1-676_jpg.rf.074926106e5d1f7d9e99c79e1f04e666
Image and label match confirmed: 1010s1120s2001-2s5300-1-886_jpg.rf.0e1c01ae629d6402ecaf54f676166fbb
Image and label match confirmed: 1010s1121s2001-2s5101-1-136_jpg.rf.1207b4095ecca7dbac948afcf9814ebf
Image and label match confirmed: 1010s1121s2001-2s5101-1-211_jpg.rf.c23c480cf8ad5706dde13cbebae229e1
Image and label match confirmed: 1010s1121s2001-2s5101-1-271_jpg.rf.11003295413988ccd4adaf

In [1]:
import os
import glob

# Define the directory where YOLO label txt files are stored
label_folder = r'C:\Users\phann\Documents\project\dataset\output_dataset_no_scale\valid'  # update to your directory

# Recursively find all txt files in the label_folder
label_files = glob.glob(os.path.join(label_folder, '**/*.txt'), recursive=True)

# Counter for images with less than 3 bounding boxes
count_images_less_than_3 = 0

for label_file in label_files:
    # Each non-empty line in a YOLO label file typically represents a bounding box.
    with open(label_file, 'r') as f:
        lines = [line for line in f if line.strip()]
    num_boxes = len(lines)
    if num_boxes < 3:
        count_images_less_than_3 += 1

print(f"Total images with less than 3 boxes: {count_images_less_than_3}")


Total images with less than 3 boxes: 0
